### ADD FEATURES TO QUINIELA

In [1]:
import os
os.getcwd()

'C:\\Users\\Albert\\Desktop\\DATA SCIENCE\\UBIQUM\\UBIQUM_1X2_Python\\notebooks'

In [2]:
import pandas as pd

xl = pd.ExcelFile('../bbdd/QUINIELA_1_CURRENT.xlsx')
df = xl.parse(xl.sheet_names[0])

### Function to add features to current Quiniela

In [3]:
# last n games of home or away team

def last_games(df, league, team, date, season, ngames):
    select = df[(df['LIGA']==league) & ((df['LOCAL']==team) | (df['VISITANTE']==team)) & 
            (df['FECHA'] < date) & (df['TEMPORADA']==season)][-ngames:]
    games = select.shape[0] + 1/2
    WINS = (sum(select['QUINIELA'][select.LOCAL==team]=='1') + sum(select['QUINIELA'][select.VISITANTE==team]=='2') + 1/6)/games
    DRAW = (sum(select['QUINIELA']=='X') + 1/6)/games
    LOSE = (1 - WINS - DRAW)
    return(WINS, DRAW, LOSE)

In [4]:
# total league. local as a local

def league_local_local(df, league, home_team, date, season):
    select = df[(df['LIGA']==league) & ((df['LOCAL']==home_team)) & 
            (df['FECHA'] < date) & (df['TEMPORADA']==season)]
    games = select.shape[0] + 1/2
    H_WINS = (sum(select['QUINIELA'][select.LOCAL==home_team]=='1') + 1/6)/games
    H_DRAW = (sum(select['QUINIELA']=='X') + 1/6)/games
    H_LOSE = (1 - H_WINS - H_DRAW)
    return(H_WINS, H_DRAW, H_LOSE)

In [5]:
# total league. local as a local

def league_local_local(df, league, home_team, date, season):
    select = df[(df['LIGA']==league) & ((df['LOCAL']==home_team)) & 
            (df['FECHA'] < date) & (df['TEMPORADA']==season)]
    games = select.shape[0] + 1/2
    H_WINS = (sum(select['QUINIELA'][select.LOCAL==home_team]=='1') + 1/6)/games
    H_DRAW = (sum(select['QUINIELA']=='X') + 1/6)/games
    H_LOSE = (1 - H_WINS - H_DRAW)
    return(H_WINS, H_DRAW, H_LOSE)

In [6]:
# total league. visitor as a visitor

def league_visitor_visitor(df, league, away_team, date, season):
    select = df[(df['LIGA']==league) & ((df['VISITANTE']==away_team)) & 
            (df['FECHA'] < date) & (df['TEMPORADA']==season)]
    games = select.shape[0] + 1/2
    V_WINS = (sum(select['QUINIELA'][select.VISITANTE==away_team]=='2') + 1/6)/games
    V_DRAW = (sum(select['QUINIELA']=='X') + 1/6)/games
    V_LOSE = (1 - V_WINS - V_DRAW)
    return(V_WINS, V_DRAW, V_LOSE)

In [7]:
# h2h last 10 games total

def h2h_10_hda(df, date, home_team, away_team):
    select = df[(df['FECHA'] < date) & (((df['LOCAL']==home_team) & (df['VISITANTE']==away_team)) |  
            ((df['LOCAL']==away_team) & (df['VISITANTE']==home_team)))][-10:]
    games = select.shape[0] + 1/2
    HOME = (sum(select['QUINIELA'][select.LOCAL==home_team]=='1') + 
            sum(select['QUINIELA'][select.VISITANTE==home_team]=='2') + 1/6)/games
    DRAW = (sum(select['QUINIELA']=='X') + 1/6)/games
    AWAY = (1 - HOME - DRAW)
    return(HOME, DRAW, AWAY)

In [8]:
# h2h last 5 games with same home and away

def h2h_5_homeaway(df, date, home_team, away_team):
    select = df[(df['FECHA'] < date) & (df['LOCAL']==home_team) & (df['VISITANTE']==away_team)][-5:]
    games = select.shape[0] + 1/2
    HOME = (sum(select['QUINIELA'][select.LOCAL==home_team]=='1') + 1/6)/games
    DRAW = (sum(select['QUINIELA'][select.LOCAL==home_team]=='X') + 1/6)/games
    AWAY = (1 - HOME - DRAW)
    return(HOME, DRAW, AWAY)

### Add features to Quiniela

In [9]:
import pandas as pd

xl = pd.ExcelFile('../bbdd/FULL_BBDD_only_results.xlsx')
historic = xl.parse(xl.sheet_names[0])

In [10]:
select_1 = list(historic[(historic['TEMPORADA']==2019) & (historic['LIGA']==1)].LOCAL.unique())
select_2 = list(historic[(historic['TEMPORADA']==2019) & (historic['LIGA']==2)].LOCAL.unique())
teams_1 = { i : 1 for i in select_1 }
teams_2 = { i : 2 for i in select_2 }
teams_league_dict = {**teams_1, **teams_2}

In [11]:
from datetime import datetime

df['LIGA'] = [teams_league_dict[i] for i in df['LOCAL']]
df['TEMPORADA'] = 2019
df['JORNADA'] = 'X'
df['FECHA'] = datetime.today().strftime('%Y-%m-%d')
df = df[['TEMPORADA', 'LIGA', 'JORNADA', 'FECHA', 'LOCAL', 'VISITANTE']]
df.head(2)

,TEMPORADA,LIGA,JORNADA,FECHA,LOCAL,VISITANTE
0,2019,1,X,2019-12-22,Mallorca,Sevilla
1,2019,1,X,2019-12-22,Villarreal,Getafe


In [12]:
df['last2_hometeam_wdl'] = df.apply(lambda x: last_games(historic, x[1], x[4], x[3], x[0], 2), axis=1)
df['last5_hometeam_wdl'] = df.apply(lambda x: last_games(historic, x[1], x[4], x[3], x[0], 5), axis=1)
df['last10_hometeam_wdl'] = df.apply(lambda x: last_games(historic, x[1], x[4], x[3], x[0], 10), axis=1)

In [13]:
df['last2_awayteam_wdl'] = df.apply(lambda x: last_games(historic, x[1], x[5], x[3], x[0], 2), axis=1)
df['last5_awayteam_wdl'] = df.apply(lambda x: last_games(historic, x[1], x[5], x[3], x[0], 5), axis=1)
df['last10_awayteam_wdl'] = df.apply(lambda x: last_games(historic, x[1], x[5], x[3], x[0], 10), axis=1)

In [14]:
df['league_local_local_wdl'] = df.apply(lambda x: league_local_local(historic, x[1], x[4], x[3], x[0]), axis=1)
df['league_visitor_visitor_wdl'] = df.apply(lambda x: league_visitor_visitor(historic, x[1], x[5], x[3], x[0]), axis=1)

In [15]:
df['h2h_last_10_hda'] = df.apply(lambda x: h2h_10_hda(historic, x[3], x[4], x[5]), axis=1)
df['h2h_5_homeaway_hda'] = df.apply(lambda x: h2h_5_homeaway(historic, x[3], x[4], x[5]), axis=1)

### Split tuples and delete columns

In [16]:
columns = list(df.columns[[x[-3:] in ['wdl','hda'] for x in df.columns]])

In [17]:
for i in columns:
    field_type = i[-3:]
    name1 = i[:-3] + field_type[0]
    name2 = i[:-3] + field_type[1]
    name3 = i[:-3] + field_type[2]
    
    df[name1] = df[i].apply(lambda x: x[0])
    df[name2] = df[i].apply(lambda x: x[1])
    df[name3] = df[i].apply(lambda x: x[2])

In [18]:
df.drop(columns, inplace=True, axis=1)

In [19]:
df.columns

Index(['TEMPORADA', 'LIGA', 'JORNADA', 'FECHA', 'LOCAL', 'VISITANTE',
       'last2_hometeam_w', 'last2_hometeam_d', 'last2_hometeam_l',
       'last5_hometeam_w', 'last5_hometeam_d', 'last5_hometeam_l',
       'last10_hometeam_w', 'last10_hometeam_d', 'last10_hometeam_l',
       'last2_awayteam_w', 'last2_awayteam_d', 'last2_awayteam_l',
       'last5_awayteam_w', 'last5_awayteam_d', 'last5_awayteam_l',
       'last10_awayteam_w', 'last10_awayteam_d', 'last10_awayteam_l',
       'league_local_local_w', 'league_local_local_d', 'league_local_local_l',
       'league_visitor_visitor_w', 'league_visitor_visitor_d',
       'league_visitor_visitor_l', 'h2h_last_10_h', 'h2h_last_10_d',
       'h2h_last_10_a', 'h2h_5_homeaway_h', 'h2h_5_homeaway_d',
       'h2h_5_homeaway_a'],
      dtype='object')

In [20]:
df

,TEMPORADA,LIGA,JORNADA,FECHA,LOCAL,VISITANTE,last2_hometeam_w,last2_hometeam_d,last2_hometeam_l,last5_hometeam_w,...,league_local_local_l,league_visitor_visitor_w,league_visitor_visitor_d,league_visitor_visitor_l,h2h_last_10_h,h2h_last_10_d,h2h_last_10_a,h2h_5_homeaway_h,h2h_5_homeaway_d,h2h_5_homeaway_a
0,2019,1,X,2019-12-22,Mallorca,Sevilla,0.066667,0.466667,0.466667,0.212121,...,0.333333,0.543860,0.228070,0.228070,0.206349,0.301587,0.492063,0.212121,0.575758,0.212121
1,2019,1,X,2019-12-22,Villarreal,Getafe,0.466667,0.466667,0.066667,0.212121,...,0.137255,0.372549,0.372549,0.254902,0.396825,0.206349,0.396825,0.575758,0.030303,0.393939
2,2019,1,X,2019-12-22,Valladolid,Valencia,0.066667,0.466667,0.466667,0.030303,...,0.155556,0.372549,0.137255,0.490196,0.111111,0.396825,0.492063,0.030303,0.393939,0.575758
3,2019,1,X,2019-12-22,Leganes,Espanyol,0.466667,0.466667,0.066667,0.212121,...,0.649123,0.254902,0.254902,0.490196,0.333333,0.025641,0.641026,0.333333,0.047619,0.619048
4,2019,1,X,2019-12-22,Osasuna,Real Sociedad,0.066667,0.466667,0.466667,0.212121,...,0.137255,0.438596,0.228070,0.333333,0.206349,0.396825,0.396825,0.393939,0.393939,0.212121
5,2019,1,X,2019-12-22,Real Betis,Atletico Madrid,0.466667,0.466667,0.066667,0.575758,...,0.228070,0.254902,0.607843,0.137255,0.111111,0.206349,0.682540,0.212121,0.212121,0.575758
6,2019,1,X,2019-12-22,Levante,Celta Vigo,0.466667,0.066667,0.466667,0.393939,...,0.254902,0.137255,0.254902,0.607843,0.206349,0.015873,0.777778,0.030303,0.030303,0.939394
7,2019,1,X,2019-12-22,Real Madrid,Athletic Club,0.466667,0.466667,0.066667,0.757576,...,0.019608,0.137255,0.490196,0.372549,0.587302,0.301587,0.111111,0.757576,0.212121,0.030303
8,2019,2,X,2019-12-22,Las Palmas,Rayo Vallecano,0.866667,0.066667,0.066667,0.757576,...,0.301587,0.206349,0.492063,0.301587,0.396825,0.206349,0.396825,0.575758,0.212121,0.212121
9,2019,2,X,2019-12-22,Cadiz,Numancia,0.466667,0.466667,0.066667,0.393939,...,0.017544,0.122807,0.543860,0.333333,0.587302,0.301587,0.111111,0.575758,0.393939,0.030303


In [22]:
df.to_excel('../bbdd/QUINIELA_2_FEATURES.xlsx', index=False)